In [2]:
import cv2
import numpy as np
from osgeo import gdal
from scipy import misc
from scipy import stats
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn import decomposition as dec
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy.linalg import orthogonal_procrustes
from sklearn.preprocessing import minmax_scale
from sklearn import cluster
from sklearn import mixture
import scipy.special as sc_special
import tkinter.filedialog
import os
from collections import Counter
import itertools as it
from sklearn import metrics
from scipy.spatial.distance import pdist, euclidean
from scipy.fftpack import dct, idct

In [11]:
#filename = tkinter.filedialog.askopenfilenames()
#%gui tk

In [1]:
filename1 = ('C:/Users/yang/Desktop/同步文件/svd分解无监督聚类/影像(show)/s2ch20190626.tif',
 'C:/Users/yang/Desktop/同步文件/svd分解无监督聚类/影像(show)/s2th20200503.tif',
 'C:/Users/yang/Desktop/同步文件/svd分解无监督聚类/影像(show)/sentinel2_th_20170529.tif',
 'C:/Users/yang/Desktop/同步文件/svd分解无监督聚类/影像(show)/sentinel2_th_20190817.tif',
 'C:/Users/yang/Desktop/同步文件/svd分解无监督聚类/影像(show)/sentinel2_th_20200503.tif',
 #'C:/Users/yang/Desktop/同步文件/svd分解无监督聚类/影像(show)/Sentinel2ch20200511.tif',
 'C:/Users/yang/Desktop/同步文件/svd分解无监督聚类/影像(show)/Sentinel2taihu20170608.tif',###
 'C:/Users/yang/Desktop/同步文件/svd分解无监督聚类/影像(show)/Sentinel2taihu20170728.tif',
 'C:/Users/yang/Desktop/同步文件/svd分解无监督聚类/影像(show)/Sentinel2taihu20180618.tif',
 'C:/Users/yang/Desktop/同步文件/svd分解无监督聚类/影像(show)/Sentinel2taihu20190603.tif',
 'C:/Users/yang/Desktop/同步文件/svd分解无监督聚类/影像(show)/Sentinel2taihu20190628.tif',
 'C:/Users/yang/Desktop/同步文件/svd分解无监督聚类/影像(show)/Sentinel2taihu20200801.tif',
 'C:/Users/yang/Desktop/同步文件/svd分解无监督聚类/影像(show)/Sentinel2taihu20200905.tif',
 'C:/Users/yang/Desktop/同步文件/svd分解无监督聚类/影像(show)/Sentinel2taihu20201005.tif',
 'C:/Users/yang/Desktop/同步文件/svd分解无监督聚类/影像(show)/Sentinel2taihu20210622.tif',
 'C:/Users/yang/Desktop/同步文件/svd分解无监督聚类/影像(show)/Sentinel2th20210324.tif',
 'C:/Users/yang/Desktop/同步文件/svd分解无监督聚类/影像(show)/Sentinel2th20211204.tif',
 'C:/Users/yang/Desktop/同步文件/svd分解无监督聚类/影像(show)/Sentinel2th20200319.tif',
 'C:/Users/yang/Desktop/同步文件/svd分解无监督聚类/影像(show)/Sentinel2th20200413.tif')

In [22]:
def jisuan_dct_svd2(path):
    ###################先定义一些函数
    def qunan(img):
        gray = img[~np.isnan(img)]
        return gray
    def Entropy(labels,base=2):   
        probs = pd.Series(labels).value_counts() / len(labels)#seriesimg
        en = stats.entropy(probs,base=base)
        return en
    def dual_thre(band):
        def cuckoo_search(n, m, fit_func, lower_boundary, upper_boundary, iter_num = 100,pa = 0.25, beta = 1.5, step_size = 0.01):
            # get initial nests' locations 
            nests = generate_nests(n, m, lower_boundary, upper_boundary)
            fitness = calc_fitness(fit_func, nests)
        
            # get the best nest and record it
            best_nest_index = np.argmax(fitness)
            best_fitness = fitness[best_nest_index]
            best_nest = nests[best_nest_index].copy()
        
            for _ in range(iter_num):
                nests = update_nests(fit_func, lower_boundary, upper_boundary, nests, best_nest, fitness, step_size)
                nests = abandon_nests(nests, lower_boundary, upper_boundary, pa)
                fitness = calc_fitness(fit_func, nests)
                
                max_nest_index = np.argmax(fitness)
                max_fitness = fitness[max_nest_index]
                max_nest = nests[max_nest_index]
        
                if (max_fitness > best_fitness):
                    best_nest = max_nest.copy()
                    best_fitness = max_fitness
        
            return (best_nest, best_fitness)

        def generate_nests(n, m, lower_boundary, upper_boundary):
            lower_boundary = np.array(lower_boundary)
            upper_boundary = np.array(upper_boundary)
            nests = np.empty((n, m))
        
            for each_nest in range(n):
                nests[each_nest] = lower_boundary + np.array([np.random.rand() for _ in range(m)]) * (upper_boundary - lower_boundary)
        
            return nests
        
        def update_nests(fit_func, lower_boundary, upper_boundary, nests, best_nest, fitness, step_coefficient):
        
            lower_boundary = np.array(lower_boundary)
            upper_boundary = np.array(upper_boundary)
            n, m = nests.shape
            # generate steps using levy flight
            steps = levy_flight(n, m, 1.5)
            new_nests = nests.copy()
        
            for each_nest in range(n):
                # coefficient 0.01 is to avoid levy flights becoming too aggresive
                # and (nest[each_nest] - best_nest) could let the best nest be remained
                step_size = step_coefficient * steps[each_nest] * (nests[each_nest] - best_nest)
                step_direction = np.random.rand(m)
                new_nests[each_nest] += step_size * step_direction
                # apply boundary condtions
                new_nests[each_nest][new_nests[each_nest] < lower_boundary] = lower_boundary[new_nests[each_nest] < lower_boundary]
                new_nests[each_nest][new_nests[each_nest] > upper_boundary] = upper_boundary[new_nests[each_nest] > upper_boundary]
        
            new_fitness = calc_fitness(fit_func, new_nests)
            nests[new_fitness > fitness] = new_nests[new_fitness > fitness]
            
            return nests
        
        def abandon_nests(nests, lower_boundary, upper_boundary, pa):
        
            lower_boundary = np.array(lower_boundary)
            upper_boundary = np.array(upper_boundary)
            n, m = nests.shape
            for each_nest in range(n):
                if (np.random.rand() < pa):
                    step_size = np.random.rand() * (nests[np.random.randint(0, n)] - nests[np.random.randint(0, n)])
                    nests[each_nest] += step_size
                    # apply boundary condtions
                    nests[each_nest][nests[each_nest] < lower_boundary] = lower_boundary[nests[each_nest] < lower_boundary]
                    nests[each_nest][nests[each_nest] > upper_boundary] = upper_boundary[nests[each_nest] > upper_boundary]
            
            return nests
        
        def levy_flight(n, m, beta):
        
            sigma_u = (sc_special.gamma(1+beta)*np.sin(np.pi*beta/2)/(sc_special.gamma((1+beta)/2)*beta*(2**((beta-1)/2))))**(1/beta)
            sigma_v = 1
        
            u =  np.random.normal(0, sigma_u, (n, m))
            v = np.random.normal(0, sigma_v, (n, m))
        
            steps = u/((np.abs(v))**(1/beta))
        
            return steps
        
        def calc_fitness(fit_func, nests):
        
            n, m = nests.shape
            fitness = np.empty(n)
        
            for each_nest in range(n):
                fitness[each_nest] = fit_func(nests[each_nest])
        
            return fitness
        if __name__=='__main__':
            
            def en(nest):
                x, y = nest  
                a = seriesimg[seriesimg<=x]
                a_shang = Entropy(a)
                b = seriesimg[(seriesimg>x)&(seriesimg<y)]
                b_shang = Entropy(b)
                c = seriesimg[seriesimg>=y]
                c_shang = Entropy(c)
                zongshang = a_shang+b_shang+c_shang
                return zongshang
            
            seriesimg = qunan(band)
            xia1,shang1,n3,n4 = cv2.minMaxLoc(seriesimg)
            #mean = np.mean(seriesimg)
            #std = np.std(seriesimg)
            #xia1 = mean-2*std
            #shang1 = mean+2*std
            best_nest1, best_fitness1 = cuckoo_search(15, 2, en, [xia1, xia1], [shang1, shang1], step_size = 0.01)
            
        return best_nest1
    def band_trunsvd(matrix,n):                
        svd = dec.TruncatedSVD(n_components=n)
        final = svd.fit_transform(matrix)
        return final
    def baocun(gray,name):
        im_width = dataset.RasterXSize    #栅格矩阵的列数
        im_height = dataset.RasterYSize   #栅格矩阵的行数
        im_geotrans = dataset.GetGeoTransform()  #仿射矩阵
        im_proj = dataset.GetProjection() #地图投影信息
        driver = gdal.GetDriverByName("GTiff")                
        dataset2 = driver.Create(name, im_width, im_height, 1, gdal.GDT_Float32)
        dataset2.SetGeoTransform(im_geotrans)              #写入仿射变换参数
        dataset2.SetProjection(im_proj)
        dataset2.GetRasterBand(1).WriteArray(gray)
        del dataset2
    
    #####################提取波段和预处理
    dataset = gdal.Open(path)    
    b=dataset.GetRasterBand(1).ReadAsArray()
    g=dataset.GetRasterBand(2).ReadAsArray()
    r=dataset.GetRasterBand(3).ReadAsArray()
    nir=dataset.GetRasterBand(4).ReadAsArray()
    swir=dataset.GetRasterBand(5).ReadAsArray()
    swir2=dataset.GetRasterBand(6).ReadAsArray()
    may_value = np.unique(b)
    if (may_value[may_value>4]).shape[0]>=100:
        b = b/10000
        g = g/10000
        r = r/10000
        nir = nir/10000
        swir = swir/10000
        swir2 = swir2/10000
    else:
        pass
    #############样点展平
    qunanband = qunan(b)
    zong = qunanband.shape[0]
    b1 = qunan(b).reshape(zong,1)
    g1 = qunan(g).reshape(zong,1)
    r1 = qunan(r).reshape(zong,1)
    nir1 = qunan(nir).reshape(zong,1)
    swir_1 = qunan(swir).reshape(zong,1)
    swir2_1 = qunan(swir2).reshape(zong,1)
    newbands1 = np.hstack((b1,g1,r1,nir1,swir_1,swir2_1))
    
    x,y = b.shape
    b2 = b.reshape(x*y,1)
    g2 = g.reshape(x*y,1)
    r2 = r.reshape(x*y,1)
    nir2 = nir.reshape(x*y,1)
    swir_2 = swir.reshape(x*y,1)
    swir2_2 = swir2.reshape(x*y,1)
    newimage = np.hstack((b2,g2,r2,nir2,swir_2,swir2_2))
    
    # def band_bianhuan1(matrix,n):                
    #     svd = dec.NMF(n_components=n)#.
    #     final = svd.fit_transform(matrix)
    #     return final
    tsvd_image = band_trunsvd(newbands1,4)
    dct_image = dct(tsvd_image,3,norm='ortho')
    
    newimage2 = newimage[:,0:4] 
    pixloc = np.where(newimage2>-1000) 
    transform2 = dct_image.flatten()
    length = pixloc[0].shape[0]
    for i in range(0,length):
        newimage2[pixloc[0][i],pixloc[1][i]]=transform2[i]
    dct1 = newimage2[:,0:1].reshape(x,y)
    dct2 = newimage2[:,1:2].reshape(x,y)
    dct3 = newimage2[:,2:3].reshape(x,y)
    #dct4 = newimage2[:,3:4].reshape(x,y)
    #dct5 = newimage2[:,4:5].reshape(x,y)
    ###########################
    dct2 = np.round(dct2,6)
    th_all =  dual_thre(dct2)
    print(th_all)
    th1,th2 = th_all
    example = np.copy(b)
    example[(dct2<=th1)]=dct_final_class_1
    example[(dct2>=th2)]=dct_final_class_2+2
    example[(dct2>th1)&(dct2<th2)] =4
   
    #muban1 = np.copy(image_tsvd1)
    #final_image = np.where(muban1>-10000,example,muban1)
    
    a,b = os.path.splitext(os.path.basename(path))
    name = str(th_all)
    save_path ='C:/'+a+name+'nmf4_dct(3,ortho)_12(round6)'+'.tif'
    baocun(example,save_path)

In [23]:
list(map(jisuan_dct_svd2,filename1))

C:\Users\yang\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
C:\Users\yang\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


[0.00166618 0.0126225 ]


C:\Users\yang\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
C:\Users\yang\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


[-0.00259035  0.00585193]


C:\Users\yang\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
C:\Users\yang\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


...

C:\Users\yang\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
C:\Users\yang\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


[-0.00560037 -0.00049032]


C:\Users\yang\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
C:\Users\yang\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


[-0.0054843   0.00221067]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]